## 📦 Installation Instructions

**IMPORTANT: Run the installation cell below FIRST before running any other cells in this notebook.**

### Steps:
1. **Run the installation cell below** to install all required libraries
2. **Wait for installation to complete** (may take a few minutes)
3. **(Optional but recommended)** Restart the kernel after installation
   - In Jupyter: Kernel → Restart
   - In Google Colab: Runtime → Restart runtime
4. **Proceed with running the rest of the notebook cells**

### Required Libraries:
- `mne` - EEG data processing and visualization
- `numpy` - Numerical computing
- `scipy` - Scientific computing
- `matplotlib` - Plotting library
- `seaborn` - Statistical visualization
- `ipywidgets` - Interactive widgets
- `pandas` - Data manipulation

In [ ]:
# ============================================================
# RUN THIS CELL FIRST - Install all required libraries
# ============================================================
# Uncomment the line below to install all dependencies

# !pip install mne numpy scipy matplotlib seaborn ipywidgets pandas

# ============================================================
# Alternative: Install from requirements.txt (if available)
# ============================================================
# !pip install -r requirements.txt

print("✓ Installation commands are ready!")
print("  Uncomment the pip install line above and run this cell to install dependencies.")
print("  After installation completes, restart the kernel and run the rest of the notebook.")

# EEG Data Visualization from PhysioNet

This notebook provides comprehensive visualization tools for EEG data from the PhysioNet EEG Motor Movement/Imagery Dataset.

## Dataset Information
- **Source**: [PhysioNet EEG Motor Movement/Imagery Dataset](https://physionet.org/content/eegmat/1.0.0/)
- **Format**: EDF (European Data Format)
- **Channels**: 64 EEG channels using the 10-20 system
- **Tasks**: Motor movement and imagery tasks

## Features
1. Raw EEG signal visualization
2. Power Spectral Density (PSD) analysis
3. Topographic maps
4. Time-frequency analysis (spectrograms)

## Requirements
See the installation instructions at the top of this notebook.

In [ ]:
# Import required libraries
import mne
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
from matplotlib import cm
import warnings
warnings. filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("All libraries imported successfully!")
print(f"MNE version: {mne.__version__}")

## 1. Data Loading Functions

These functions help load EEG data from PhysioNet or local files.

In [ ]:
def download_physionet_data(subject=1, run=1, path='./data'):
    """
    Download EEG data from PhysioNet database.
    
    Parameters:
    -----------
    subject : int
        Subject number (1-109)
    run : int
        Run number (1-14)
    path : str
        Directory to save downloaded data
    
    Returns:
    --------
    raw :  mne.io.Raw
        Raw EEG data object
    """
    print(f"Downloading data for Subject {subject: 03d}, Run {run:02d}... ")
    
    # PhysioNet URL structure
    url = f'https://physionet.org/files/eegmmidb/1.0.0/S{subject:03d}/S{subject:03d}R{run: 02d}. edf'
    
    try:
        # Load EDF file
        raw = mne.io.read_raw_edf(url, preload=True, verbose=False)
        print(f"✓ Data loaded successfully! ")
        return raw
    except Exception as e:
        print(f"✗ Error loading data: {e}")
        return None

def load_local_edf(filepath):
    """
    Load EEG data from a local EDF file.
    
    Parameters:
    -----------
    filepath : str
        Path to the EDF file
    
    Returns:
    --------
    raw : mne.io.Raw
        Raw EEG data object
    """
    try:
        raw = mne.io.read_raw_edf(filepath, preload=True, verbose=False)
        print(f"✓ File loaded successfully:  {filepath}")
        return raw
    except Exception as e:
        print(f"✗ Error loading file: {e}")
        return None

def display_data_info(raw):
    """
    Display basic information about the loaded EEG data.
    
    Parameters:
    -----------
    raw :  mne.io.Raw
        Raw EEG data object
    """
    print("\n" + "="*60)
    print("EEG DATA INFORMATION")
    print("="*60)
    print(f"Number of channels: {len(raw.ch_names)}")
    print(f"Sampling frequency: {raw.info['sfreq']} Hz")
    print(f"Duration: {raw.times[-1]:. 2f} seconds")
    print(f"Number of samples: {len(raw. times)}")
    print(f"\nChannel names:")
    for i, ch in enumerate(raw.ch_names):
        print(f"  {i+1: 2d}. {ch}")
    print("="*60 + "\n")

## 2. Load Sample Data

Modify the parameters below to load different subjects or runs.

**Run descriptions:**
- Run 1: Baseline, eyes open
- Run 2: Baseline, eyes closed
- Run 3-14: Various motor movement and imagery tasks

In [ ]:
# ============= MODIFY THESE PARAMETERS =============
SUBJECT = 1      # Subject number (1-109)
RUN = 3          # Run number (1-14)
# ==================================================

# Load data from PhysioNet
raw = download_physionet_data(subject=SUBJECT, run=RUN)

# Uncomment the line below to load from a local file instead
# raw = load_local_edf('path/to/your/file.edf')

if raw is not None:
    display_data_info(raw)

## 3. Raw EEG Signal Visualization

Visualize the raw EEG time series data.

In [ ]:
def plot_raw_eeg(raw, start=0, duration=10, n_channels=10, scalings='auto'):
    """
    Plot raw EEG signals.
    
    Parameters:
    -----------
    raw : mne.io.Raw
        Raw EEG data object
    start : float
        Start time in seconds
    duration : float
        Duration to plot in seconds
    n_channels : int
        Number of channels to display
    scalings : str or dict
        Scaling factor for the data
    """
    fig = raw.plot(start=start, duration=duration, n_channels=n_channels, 
                   scalings=scalings, show=True, block=False)
    plt.suptitle(f'Raw EEG Signals (Time:  {start}-{start+duration}s)', 
                 fontsize=14, fontweight='bold')
    return fig

# ============= MODIFY PLOTTING PARAMETERS =============
START_TIME = 0      # Start time in seconds
DURATION = 10       # Duration in seconds
N_CHANNELS = 10     # Number of channels to display
# =====================================================

if raw is not None:
    plot_raw_eeg(raw, start=START_TIME, duration=DURATION, n_channels=N_CHANNELS)

## 4. Power Spectral Density (PSD) Analysis

Analyze the frequency content of EEG signals using Power Spectral Density.

In [ ]:
def plot_psd(raw, fmin=0.5, fmax=50, tmin=0, tmax=None):
    """
    Plot Power Spectral Density for all channels.
    
    Parameters:
    -----------
    raw : mne.io.Raw
        Raw EEG data object
    fmin : float
        Minimum frequency to display
    fmax : float
        Maximum frequency to display
    tmin :  float
        Start time for PSD calculation
    tmax : float
        End time for PSD calculation (None = end of recording)
    """
    # Compute PSD
    fig = raw.compute_psd(fmin=fmin, fmax=fmax, tmin=tmin, tmax=tmax).plot(
        average=False, picks='eeg', exclude='bads', amplitude=False
    )
    
    plt.suptitle('Power Spectral Density - All Channels', 
                 fontsize=14, fontweight='bold')
    
    # Add frequency band annotations
    ax = plt.gca()
    bands = {'Delta': (0.5, 4), 'Theta': (4, 8), 'Alpha': (8, 13), 
             'Beta': (13, 30), 'Gamma': (30, 50)}
    
    for band, (low, high) in bands.items():
        ax.axvspan(low, high, alpha=0.1, label=band)
    
    return fig

def plot_psd_topomap(raw, bands=None):
    """
    Plot topographic maps of PSD for different frequency bands.
    
    Parameters:
    -----------
    raw : mne.io.Raw
        Raw EEG data object
    bands : dict
        Dictionary of frequency bands {name: (fmin, fmax)}
    """
    if bands is None:
        bands = {'Delta': (0.5, 4), 'Theta': (4, 8), 'Alpha': (8, 13), 
                 'Beta': (13, 30), 'Gamma': (30, 50)}
    
    spectrum = raw.compute_psd(picks='eeg')
    
    fig, axes = plt.subplots(1, len(bands), figsize=(15, 3))
    
    for idx, (band_name, (fmin, fmax)) in enumerate(bands.items()):
        spectrum. plot_topomap(bands=[(fmin, fmax, band_name)], 
                             axes=axes[idx], show=False)
    
    plt.suptitle('PSD Topographic Maps by Frequency Band', 
                 fontsize=14, fontweight='bold', y=1.05)
    plt.tight_layout()
    return fig

# Plot PSD
if raw is not None:
    print("Generating PSD plots...")
    plot_psd(raw, fmin=0.5, fmax=50)
    plt.show()
    
    print("Generating PSD topographic maps...")
    plot_psd_topomap(raw)
    plt. show()

## 5. Topographic Maps

Visualize spatial distribution of brain activity across the scalp.

In [ ]:
def plot_topomap_at_time(raw, times=[1, 2, 3, 4, 5]):
    """
    Plot topographic maps at specific time points.
    
    Parameters:
    -----------
    raw : mne.io. Raw
        Raw EEG data object
    times : list
        List of time points (in seconds) to plot
    """
    # Get data and create evoked object for topomap plotting
    data = raw.get_data(picks='eeg')
    info = mne.pick_info(raw. info, mne.pick_types(raw.info, eeg=True))
    
    fig, axes = plt.subplots(1, len(times), figsize=(15, 3))
    
    if len(times) == 1:
        axes = [axes]
    
    for idx, t in enumerate(times):
        # Get sample index
        sample_idx = int(t * raw. info['sfreq'])
        
        # Plot topomap
        mne.viz.plot_topomap(data[: , sample_idx], info, axes=axes[idx], 
                            show=False, contours=6, cmap='RdBu_r')
        axes[idx].set_title(f't = {t}s', fontsize=12)
    
    plt.suptitle('Topographic Maps at Different Time Points', 
                 fontsize=14, fontweight='bold', y=1.05)
    plt.tight_layout()
    return fig

# ============= MODIFY TIME POINTS =============
TIME_POINTS = [1, 2, 3, 4, 5]  # Time points in seconds
# =============================================

if raw is not None:
    print("Generating topographic maps... ")
    plot_topomap_at_time(raw, times=TIME_POINTS)
    plt.show()

## 6. Time-Frequency Analysis (Spectrograms)

Visualize how frequency content changes over time using spectrograms.

In [ ]:
def plot_spectrogram(raw, channel='C3', tmin=0, tmax=None, fmin=0.5, fmax=50):
    """
    Plot time-frequency spectrogram for a specific channel.
    
    Parameters:
    -----------
    raw : mne.io. Raw
        Raw EEG data object
    channel : str
        Channel name to plot
    tmin : float
        Start time in seconds
    tmax : float
        End time in seconds (None = end of recording)
    fmin : float
        Minimum frequency to display
    fmax : float
        Maximum frequency to display
    """
    if channel not in raw.ch_names:
        print(f"Channel '{channel}' not found.  Available channels: {raw.ch_names}")
        return
    
    # Get data for the specified channel
    data, times = raw[channel, :]
    data = data.flatten()
    
    # Apply time window
    if tmax is None:
        tmax = times[-1]
    
    start_idx = int(tmin * raw.info['sfreq'])
    end_idx = int(tmax * raw.info['sfreq'])
    data = data[start_idx:end_idx]
    times = times[start_idx:end_idx]
    
    # Compute spectrogram
    frequencies, times_spec, Sxx = signal.spectrogram(
        data, raw.info['sfreq'], nperseg=int(2*raw.info['sfreq']),
        noverlap=int(1.5*raw.info['sfreq'])
    )
    
    # Filter frequency range
    freq_mask = (frequencies >= fmin) & (frequencies <= fmax)
    frequencies = frequencies[freq_mask]
    Sxx = Sxx[freq_mask, :]
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))
    im = ax.pcolormesh(times_spec + tmin, frequencies, 10 * np.log10(Sxx), 
                       shading='gouraud', cmap='jet')
    ax.set_ylabel('Frequency (Hz)', fontsize=12)
    ax.set_xlabel('Time (s)', fontsize=12)
    ax.set_title(f'Spectrogram - Channel {channel}', fontsize=14, fontweight='bold')
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Power (dB)', fontsize=12)
    
    # Add frequency band lines
    bands = {'Delta': 4, 'Theta': 8, 'Alpha': 13, 'Beta': 30}
    for band, freq in bands.items():
        if fmin <= freq <= fmax:
            ax.axhline(y=freq, color='white', linestyle='--', linewidth=1, alpha=0.5)
            ax.text(times_spec[-1] + tmin, freq, f' {band}', 
                   color='white', fontsize=9, va='center')
    
    plt.tight_layout()
    return fig

def plot_multiple_spectrograms(raw, channels=['C3', 'C4', 'Cz'], 
                               tmin=0, tmax=None, fmin=0.5, fmax=50):
    """
    Plot spectrograms for multiple channels.
    
    Parameters:
    -----------
    raw : mne. io.Raw
        Raw EEG data object
    channels : list
        List of channel names to plot
    tmin : float
        Start time in seconds
    tmax : float
        End time in seconds
    fmin : float
        Minimum frequency
    fmax : float
        Maximum frequency
    """
    # Filter valid channels
    valid_channels = [ch for ch in channels if ch in raw.ch_names]
    
    if not valid_channels:
        print(f"None of the specified channels found.  Available:  {raw.ch_names[: 10]}... ")
        return
    
    fig, axes = plt.subplots(len(valid_channels), 1, 
                             figsize=(12, 4*len(valid_channels)))
    
    if len(valid_channels) == 1:
        axes = [axes]
    
    for idx, channel in enumerate(valid_channels):
        # Get data
        data, times = raw[channel, :]
        data = data.flatten()
        
        # Apply time window
        if tmax is None:
            tmax_use = times[-1]
        else:
            tmax_use = tmax
        
        start_idx = int(tmin * raw.info['sfreq'])
        end_idx = int(tmax_use * raw.info['sfreq'])
        data = data[start_idx:end_idx]
        
        # Compute spectrogram
        frequencies, times_spec, Sxx = signal.spectrogram(
            data, raw.info['sfreq'], nperseg=int(2*raw.info['sfreq']),
            noverlap=int(1.5*raw.info['sfreq'])
        )
        
        # Filter frequency range
        freq_mask = (frequencies >= fmin) & (frequencies <= fmax)
        frequencies = frequencies[freq_mask]
        Sxx = Sxx[freq_mask, : ]
        
        # Plot
        im = axes[idx].pcolormesh(times_spec + tmin, frequencies, 
                                 10 * np.log10(Sxx), shading='gouraud', cmap='jet')
        axes[idx].set_ylabel('Frequency (Hz)', fontsize=11)
        axes[idx].set_title(f'Channel {channel}', fontsize=12, fontweight='bold')
        
        # Add colorbar
        cbar = plt.colorbar(im, ax=axes[idx])
        cbar.set_label('Power (dB)', fontsize=10)
    
    axes[-1].set_xlabel('Time (s)', fontsize=11)
    plt.suptitle('Time-Frequency Spectrograms', fontsize=14, fontweight='bold')
    plt.tight_layout()
    return fig

# ============= MODIFY PARAMETERS =============
CHANNEL = 'C3'                    # Single channel for detailed view
CHANNELS_MULTI = ['C3', 'C4', 'Cz']  # Multiple channels
TMIN = 0                          # Start time
TMAX = 30                         # End time (None = all data)
FMIN = 0.5                        # Min frequency
FMAX = 50                         # Max frequency
# ============================================

if raw is not None:
    print(f"Generating spectrogram for channel {CHANNEL}...")
    plot_spectrogram(raw, channel=CHANNEL, tmin=TMIN, tmax=TMAX, 
                    fmin=FMIN, fmax=FMAX)
    plt.show()
    
    print(f"\nGenerating spectrograms for channels {CHANNELS_MULTI}...")
    plot_multiple_spectrograms(raw, channels=CHANNELS_MULTI, tmin=TMIN, 
                              tmax=TMAX, fmin=FMIN, fmax=FMAX)
    plt.show()

## 7. Complete Analysis Pipeline

Run all visualizations in one go.

In [ ]:
def complete_eeg_analysis(raw, subject_id='Unknown', run_id='Unknown'):
    """
    Perform complete EEG analysis with all visualization types.
    
    Parameters:
    -----------
    raw :  mne.io.Raw
        Raw EEG data object
    subject_id : str
        Subject identifier
    run_id : str
        Run identifier
    """
    print("="*60)
    print(f"COMPLETE EEG ANALYSIS")
    print(f"Subject: {subject_id} | Run: {run_id}")
    print("="*60)
    
    # 1. Display info
    display_data_info(raw)
    
    # 2. Raw signals
    print("\n[1/5] Plotting raw EEG signals...")
    plot_raw_eeg(raw, duration=10, n_channels=10)
    plt.show()
    
    # 3. PSD
    print("\n[2/5] Computing Power Spectral Density...")
    plot_psd(raw)
    plt. show()
    
    # 4. PSD Topomaps
    print("\n[3/5] Generating PSD topographic maps...")
    plot_psd_topomap(raw)
    plt.show()
    
    # 5. Topomaps at time points
    print("\n[4/5] Creating topographic maps... ")
    plot_topomap_at_time(raw, times=[1, 3, 5, 7, 9])
    plt.show()
    
    # 6. Spectrograms
    print("\n[5/5] Generating time-frequency spectrograms...")
    plot_multiple_spectrograms(raw, channels=['C3', 'C4', 'Cz'], tmax=30)
    plt.show()
    
    print("\n" + "="*60)
    print("ANALYSIS COMPLETE! ")
    print("="*60)

# Run complete analysis
if raw is not None:
    complete_eeg_analysis(raw, subject_id=f"S{SUBJECT: 03d}", run_id=f"R{RUN:02d}")

## 8. Custom Analysis

Use this section for your own custom analyses and visualizations.

In [ ]:
# Your custom code here
# Example: Filter data and compare

if raw is not None:
    # Create a copy and apply bandpass filter
    raw_filtered = raw.copy().filter(l_freq=8, h_freq=13, picks='eeg')
    
    print("Alpha band (8-13 Hz) filtered data:")
    plot_raw_eeg(raw_filtered, duration=10, n_channels=5)
    plt.show()

## Summary

This notebook provides a comprehensive toolkit for EEG data visualization including:

1. **Data Loading**:  Download from PhysioNet or load local EDF files
2. **Raw Signals**: Time-series visualization of EEG channels
3. **Power Spectral Density**:  Frequency domain analysis
4. **Topographic Maps**: Spatial distribution of brain activity
5. **Time-Frequency Analysis**: Spectrograms showing dynamic frequency content

### Next Steps:
- Experiment with different subjects and runs
- Apply preprocessing (filtering, artifact removal)
- Perform event-related potential (ERP) analysis
- Compare motor movement vs imagery tasks
- Implement machine learning classifiers

### References:
- [MNE-Python Documentation](https://mne.tools/)
- [PhysioNet EEG Dataset](https://physionet.org/content/eegmmidb/1.0.0/)
- [10-20 Electrode System](https://en.wikipedia.org/wiki/10%E2%80%9320_system_(EEG))